In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../csv/train.csv')

In [5]:
df[:10]

,Unnamed: 0,image,label
0,0,NORMAL2-IM-1020-0001.jpeg,NORMAL
1,1,NORMAL2-IM-0843-0001.jpeg,NORMAL
2,2,NORMAL2-IM-0866-0001.jpeg,NORMAL
3,3,IM-0543-0001.jpeg,NORMAL
4,4,IM-0131-0001.jpeg,NORMAL
5,5,NORMAL2-IM-0836-0001.jpeg,NORMAL
6,6,NORMAL2-IM-0705-0001.jpeg,NORMAL
7,7,IM-0505-0001-0001.jpeg,NORMAL
8,8,IM-0663-0001.jpeg,NORMAL
9,9,NORMAL2-IM-0687-0001.jpeg,NORMAL


In [21]:
len(df)

5216

In [11]:
df_len = len(df)
individual_len = int(df_len/3)
df_list = [df[individual_len*i:individual_len*(i+1)] for i in range(3)]

In [20]:
dist = [0.3, 0.5, 0.2]

In [22]:
data_distribution = [int(df_len*ratio) for ratio in dist]

In [23]:
data_distribution

[1564, 2608, 1043]

In [27]:
for idx in range(1,len(data_distribution)):
    data_distribution[idx] = data_distribution[idx]+data_distribution[idx-1]

In [28]:
data_distribution

[1564, 4172, 5215]

In [32]:
data_distribution.insert(0, 0)

In [33]:
data_distribution

[0, 1564, 4172, 5215]

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import os
from PIL import Image
import numpy as np
import PIL

In [44]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

In [45]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [46]:
model = Net()

In [50]:
model1 = model
model2 = model
model1.send(bob)
model2.send(alice)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [54]:
model1.send(bob)

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)

In [41]:
model1

Net(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=800, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)